#Fake news  classification

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Task1 : Loading and Exploring the dataset

In [ ]:
import pandas as pd

# Path to your CSV file in Drive
file_path_true = '/content/drive/My Drive/News/True.csv'
file_path_fake = '/content/drive/My Drive/News/Fake.csv'

# Load the CSV
df_true = pd.read_csv(file_path_true)
df_fake = pd.read_csv(file_path_fake)

# Display the first few rows of the DataFrame
df_true.head()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


#Labeling dataset

In [ ]:
df_true['News_Type'] = 1
df_fake['News_Type'] = 0
df_true.head()


,title,text,subject,date,News_Type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


#Combine the two DataFrames into a single dataset.

In [ ]:
df_news=pd.concat([df_true,df_fake],axis=0,ignore_index=True)
df_news.head()
df_shuffled=df_news.sample(frac=1,random_state=42).reset_index(drop=True)
df_shuffled.to_csv('/content/drive/My Drive/News/news_dataset.csv',index=False)

In [ ]:
df_shuffled.head()

,title,text,subject,date,News_Type
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
print(df_shuffled.dtypes)


title        object
text         object
subject      object
date         object
News_Type     int64
dtype: object


In [ ]:
# Display the first few rows of the key features
import pandas as pd
print(df_shuffled[['title', 'text', 'News_Type']].head())


                                               title  \
0   BREAKING: GOP Chairman Grassley Has Had Enoug...   
1   Failed GOP Candidates Remembered In Hilarious...   
2   Mike Pence’s New DC Neighbors Are HILARIOUSLY...   
3  California AG pledges to defend birth control ...   
4  AZ RANCHERS Living On US-Mexico Border Destroy...   

                                                text  News_Type  
0  Donald Trump s White House is in chaos, and th...          0  
1  Now that Donald Trump is the presumptive GOP n...          0  
2  Mike Pence is a huge homophobe. He supports ex...          0  
3  SAN FRANCISCO (Reuters) - California Attorney ...          1  
4  Twisted reasoning is all that comes from Pelos...          0  


#Text 2:Preprocessing the textual data

In [ ]:
df_shuffled.duplicated().sum()

np.int64(209)

In [ ]:
df_shuffled.drop_duplicates(inplace=True)
df_shuffled.shape

(44689, 5)

drop any rows with missing values to ensure data integrity.

In [ ]:
df_shuffled.isnull().sum()
df_shuffled.dropna(inplace=True)

Merge the title and text columns into a single column named “content”

In [ ]:
df_shuffled['content']=df_shuffled['title']+' '+df_shuffled['text']
df_shuffled['content']=df_shuffled['content'].apply(lambda x: x.lower())
df_shuffled.head()

,title,text,subject,date,News_Type,content
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0,breaking: gop chairman grassley has had enoug...
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0,failed gop candidates remembered in hilarious...
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0,mike pence’s new dc neighbors are hilariously...
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1,california ag pledges to defend birth control ...
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0,az ranchers living on us-mexico border destroy...


In [ ]:
print(df_shuffled[['title', 'text', 'content']].head())


                                               title  \
0   BREAKING: GOP Chairman Grassley Has Had Enoug...   
1   Failed GOP Candidates Remembered In Hilarious...   
2   Mike Pence’s New DC Neighbors Are HILARIOUSLY...   
3  California AG pledges to defend birth control ...   
4  AZ RANCHERS Living On US-Mexico Border Destroy...   

                                                text  \
0  Donald Trump s White House is in chaos, and th...   
1  Now that Donald Trump is the presumptive GOP n...   
2  Mike Pence is a huge homophobe. He supports ex...   
3  SAN FRANCISCO (Reuters) - California Attorney ...   
4  Twisted reasoning is all that comes from Pelos...   

                                             content  
0   BREAKING: GOP Chairman Grassley Has Had Enoug...  
1   Failed GOP Candidates Remembered In Hilarious...  
2   Mike Pence’s New DC Neighbors Are HILARIOUSLY...  
3  California AG pledges to defend birth control ...  
4  AZ RANCHERS Living On US-Mexico Border Destroy..

#Task 3: Extracting features and splitting the dataset

In [ ]:
X=df_shuffled['content']
y=df_shuffled['News_Type']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
y

,News_Type
0,0
1,0
2,0
3,1
4,0
...,...
44892,1
44894,0
44895,0
44896,1


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(max_features=5000)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

#Task 4: Training and evaluating the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test,y_pred)

0.9881405236070709

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4724
           1       0.99      0.99      0.99      4214

    accuracy                           0.99      8938
   macro avg       0.99      0.99      0.99      8938
weighted avg       0.99      0.99      0.99      8938

